# 베이스 만들기



In [1]:
import pandas as pd
df= pd.read_csv('zomato_dataset.csv')

In [2]:
#베이스로 만들기
df.dropna(inplace=True)

In [3]:
feature_cols = ['Dining_Rating','Delivery_Rating','Dining Votes','Votes']
label = df.pop('Prices')
feature = df[feature_cols].copy()

## 회귀분석
- price를 예측하는 것

In [4]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(feature, label,  test_size=0.2, random_state=111)

In [6]:
from sklearn.preprocessing import StandardScaler
import numpy as np

scaler =StandardScaler()
train_x_sc =scaler.fit_transform(train_x)
test_x_sc = scaler.transform(test_x)
#np.reshape 조정하기

train_y =np.array(train_y)
train_y =train_y.reshape(-1,1)
test_y = np.array(test_y)
test_y = test_y.reshape(-1,1)

train_y_sc = scaler.fit_transform(train_y)
test_y_sc = scaler.transform(test_y)

## 베이스모델로 간단한 딥러닝
- 단순하게 순전파 모델 하나 만들어서 모델링 진행할 예정
- 복잡하게 역전파 만들지 않고 tensorflow로 가볍게 순전파만 진행할 예정
- Dense도 적당하게

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


model = Sequential([
    Dense(units=32, activation='relu', input_shape=[4]),
    Dense(units=16, activation='relu',),
    Dense(units=1)
])

model.compile(loss='mae',
             optimizer='adam',
             metrics=['mae','mse'])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                160       
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 705
Trainable params: 705
Non-trainable params: 0
_________________________________________________________________


In [9]:
train_x

,Dining_Rating,Delivery_Rating,Dining Votes,Votes
53177,3.6,4.2,8,0
102659,4.0,3.7,443,451
10678,3.8,4.3,146,397
8141,4.2,4.3,221,95
36951,3.8,3.7,0,0
...,...,...,...,...
51931,3.4,3.9,427,133
31916,3.4,3.7,127,13
35966,4.4,3.8,349,0
69158,3.6,3.8,0,0


In [10]:
%%time
model.fit(train_x_sc, train_y_sc, epochs=50)

Epoch 1/50
506/506 [==============================] - 3s 4ms/step - loss: 0.5739 - mae: 0.5739 - mse: 0.9951
Epoch 2/50
506/506 [==============================] - 2s 3ms/step - loss: 0.5604 - mae: 0.5604 - mse: 0.9630
Epoch 3/50
506/506 [==============================] - 1s 2ms/step - loss: 0.5556 - mae: 0.5556 - mse: 0.9478
Epoch 4/50
506/506 [==============================] - 1s 2ms/step - loss: 0.5519 - mae: 0.5519 - mse: 0.9396
Epoch 5/50
506/506 [==============================] - 1s 2ms/step - loss: 0.5488 - mae: 0.5488 - mse: 0.9334
Epoch 6/50
506/506 [==============================] - 1s 2ms/step - loss: 0.5458 - mae: 0.5458 - mse: 0.9294
Epoch 7/50
506/506 [==============================] - 1s 2ms/step - loss: 0.5443 - mae: 0.5443 - mse: 0.9281
Epoch 8/50
506/506 [==============================] - 1s 2ms/step - loss: 0.5424 - mae: 0.5424 - mse: 0.9258
Epoch 9/50
506/506 [==============================] - 1s 2ms/step - loss: 0.5407 - mae: 0.5407 - mse: 0.9195
Epoch 10/50
506/506

In [15]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

lr = LinearRegression()
lr.fit(train_x_sc,train_y_sc)
y_preds = lr.predict(test_x_sc)
mse = mean_squared_error(test_y_sc, y_preds)
rmse = np.sqrt(mse)

print(mse, rmse)
print(r2_score(test_y_sc, y_preds))

0.9844946724033561 0.9922170490388462
0.013277998034848859


### 결과: 34830.2148
- base라인 모델 - 34830.2148
- 스케일링을 진행한 경우 ( 피처와 y값 모두 standard)

# Mine 모델로 진행

In [24]:
df1= pd.read_csv('/content/0806_zomato (1).csv')
df1

,Item_Enc,Restaurant_Name_Enc,Cuisine_Enc,Place_Enc,City_Enc,MinMax_Dining_Rating,MinMax_Delivery_Rating,Robust_Delivery_Votes,Robust_Dining_Votes,Robust_Votes,Prices
0,23575,563,13,142,4,0.608696,0.809524,0.00000,0.041475,5.600000,249.0
1,44773,563,13,142,4,0.608696,0.809524,0.00000,0.041475,3.000000,129.0
2,33505,563,13,142,4,0.608696,0.809524,0.00000,0.041475,2.600000,189.0
3,33278,563,13,142,4,0.608696,0.809524,0.00000,0.041475,2.866667,189.0
4,30157,563,13,142,4,0.608696,0.809524,0.00000,0.041475,2.066667,205.0
...,...,...,...,...,...,...,...,...,...,...,...
123652,5455,75,30,68,16,0.608696,0.809524,22.73913,-0.078341,0.000000,525.0
123653,3877,75,30,68,16,0.608696,0.809524,22.73913,-0.078341,0.000000,525.0
123654,3842,75,30,68,16,0.608696,0.809524,22.73913,-0.078341,0.000000,525.0
123655,3844,75,30,68,16,0.608696,0.809524,22.73913,-0.078341,0.000000,525.0


In [25]:
df1.columns

Index(['Item_Enc', 'Restaurant_Name_Enc', 'Cuisine_Enc', 'Place_Enc',
       'City_Enc', 'MinMax_Dining_Rating', 'MinMax_Delivery_Rating',
       'Robust_Delivery_Votes', 'Robust_Dining_Votes', 'Robust_Votes',
       'Prices'],
      dtype='object')

In [26]:
feature_cols = ['Item_Enc', 'Restaurant_Name_Enc', 'Cuisine_Enc', 'Place_Enc',
       'City_Enc', 'MinMax_Dining_Rating', 'MinMax_Delivery_Rating',
       'Robust_Delivery_Votes', 'Robust_Dining_Votes', 'Robust_Votes',]
label1 = df1.pop('Prices')
feature1 = df1[feature_cols].copy()

from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(feature1, label1,  test_size=0.2, random_state=111)

In [29]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model1 = Sequential([
    Dense(units=32, activation='relu', input_shape=[10]),
    Dense(units=16, activation='relu',),
    Dense(units=1) ])

model1.compile(loss='mae',
             optimizer='adam',
             metrics=['mae','mse'])

## 학습

In [30]:
%%time
model1.fit(train_x, train_y, epochs=50)

Epoch 1/50
3092/3092 [==============================] - 9s 3ms/step - loss: 144.4586 - mae: 144.4586 - mse: 44381.5742
Epoch 2/50
3092/3092 [==============================] - 7s 2ms/step - loss: 114.2852 - mae: 114.2852 - mse: 25851.6602
Epoch 3/50
3092/3092 [==============================] - 8s 3ms/step - loss: 104.0086 - mae: 104.0086 - mse: 21087.2949
Epoch 4/50
3092/3092 [==============================] - 8s 3ms/step - loss: 96.9375 - mae: 96.9375 - mse: 18250.6152
Epoch 5/50
3092/3092 [==============================] - 7s 2ms/step - loss: 86.4537 - mae: 86.4537 - mse: 14629.0312
Epoch 6/50
3092/3092 [==============================] - 8s 3ms/step - loss: 76.0260 - mae: 76.0260 - mse: 11384.3672
Epoch 7/50
3092/3092 [==============================] - 8s 2ms/step - loss: 67.4994 - mae: 67.4994 - mse: 9271.4082
Epoch 8/50
3092/3092 [==============================] - 8s 3ms/step - loss: 59.8646 - mae: 59.8646 - mse: 7321.5869
Epoch 9/50
3092/3092 [==============================] - 8s 3

In [31]:
lr = LinearRegression()
lr.fit(train_x,train_y)
y_preds = lr.predict(test_x)
mse = mean_squared_error(test_y, y_preds)
rmse = np.sqrt(mse)

print(mse, rmse)
print(r2_score(test_y, y_preds))

20942.97216070111 144.71686895694333
0.5045013355585621


In [32]:
train_y

118638    545.00
103553     20.00
49816     280.00
90693     449.00
56339     343.00
           ...  
105747    330.00
102486    330.00
4820      285.00
10196     320.00
77652     103.81
Name: Prices, Length: 98925, dtype: float64

- mse가 큰 이유는 스케일링이 안되어서!
- 인코딩을 하고 나서도 스케일링 해주어야한다!